# Quick Start

Below is a sample demo of interaction with the environment.

In [1]:
from maro.simulator import Env
from maro.simulator.scenarios.ecr.common import Action, DecisionEvent

env = Env(scenario="ecr", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

metrics: object = None
decision_event: DecisionEvent = None
is_done: bool = False

while not is_done:
    action: Action = None
    metrics, decision_event, is_done = env.step(action)

print(metrics)

{'perf': 0.5, 'total_shortage': 100000, 'total_cost': 0}


# Environment of ECR

To initialize an environment, you need to specify the values of several parameters:
- **scenario**: The target scenario of this Env. "ecr" denotes for the Empty Container Repositioning (ECR).
- **topology**: The target topology of this Env.
   - There are some predefined topologies in MARO, that you can directly use it as in the demo.
   - Also, you can define your own topologies following the guidance in the [doc](docs/customization/new_topology.rst).  #TODO
- **start_tick**: The start tick of this Env, 1 tick corresponds to 1 day in ecr.
   - In the demo above, *start_tick=0* indicates a simulation start from the beginning of the given topology.
- **durations**: The duration of thie Env, in the unit of tick/day.
   - In the demo above, *durations=100* indicates a simulation length of 100 days.

You can get all available scenarios and topologies by calling:

In [2]:
from maro.simulator.utils import get_scenarios, get_topologies
from pprint import pprint
from typing import List

scenarios: List[str] = get_scenarios()
topologies: List[str] = get_topologies(scenario='ecr')

pprint(f'The available scenarios in MARO:')
pprint(scenarios)

print()
pprint(f'The predefined topologies in ECR:')             # TODO: update the ordered output
pprint(topologies)

'The available scenarios in MARO:'
['citi_bike', 'ecr']

'The predefined topologies in ECR:'
['toy.6p_sssbdd_l0.3',
 'toy.4p_ssdd_l0.2',
 'global_trade.22p_l0.7',
 'toy.5p_ssddd_l0.1',
 'toy.6p_sssbdd_l0.1',
 'toy.5p_ssddd_l0.8',
 'toy.6p_sssbdd_l0.8',
 'toy.6p_sssbdd_l0.2',
 'toy.6p_sssbdd_l0.5',
 'toy.6p_sssbdd_l0.0',
 'toy.6p_sssbdd_l0.7',
 'toy.5p_ssddd_l0.3',
 'toy.4p_ssdd_l0.3',
 'toy.5p_ssddd_l0.5',
 'toy.4p_ssdd_l0.8',
 'toy.4p_ssdd_l0.1',
 'toy.6p_sssbdd_l0.4',
 'toy.5p_ssddd_l0.7',
 'global_trade.22p_l0.3',
 'global_trade.22p_l0.8',
 'global_trade.22p_l0.2',
 'toy.4p_ssdd_l0.4',
 'toy.6p_sssbdd_l0.6',
 'toy.4p_ssdd_l0.7',
 'toy.5p_ssddd_l0.4',
 'global_trade.22p_l0.1',
 'global_trade.22p_l0.0',
 'global_trade.22p_l0.6',
 'toy.4p_ssdd_l0.5',
 'toy.5p_ssddd_l0.2',
 'toy.4p_ssdd_l0.0',
 'global_trade.22p_l0.5',
 'toy.5p_ssddd_l0.0',
 'toy.5p_ssddd_l0.6',
 'global_trade.22p_l0.4',
 'toy.4p_ssdd_l0.6']


Once you created an instance of the environment, you can easily access the real-time information of this environment, like:

In [3]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for ECR scenario
env = Env(scenario="ecr", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

# The current tick
tick: int = env.tick
print(f"The current tick: {tick}.")

# The current frame index, which indicates the index of current frame in the snapshot-list
frame_index: int = env.frame_index
print(f"The current frame index: {frame_index}.")

# The agent index list in the environment
agent_idx_list: List[int] = env.agent_idx_list
print(f"There are {len(agent_idx_list)} agents in this Env.")

# The whole snapshot-list of the environment, snapshots are taken in the granularity of the given snapshot_resolution
# The example of how to use the snapshot will be shown later
snapshot_list: SnapshotList = env.snapshot_list
print(f"There will be {len(snapshot_list)} snapshots in total.")

# The summary info of the environment
summary: dict = env.summary
print(f"\nEnv Summary:")
pprint(summary)

# The metrics of the environment
metrics: dict = env.metrics
print(f"\nEnv Metrics:")
pprint(metrics)

The current tick: 0.
The current frame index: 0.
There are 5 agents in this Env.
There will be 100 snapshots in total.

Env Summary:
{'node_detail': {'matrices': {'attributes': {'full_on_ports': {'slots': 25,
                                                               'type': 'i'},
                                             'full_on_vessels': {'slots': 30,
                                                                 'type': 'i'},
                                             'vessel_plans': {'slots': 30,
                                                              'type': 'i'}},
                              'number': 1},
                 'ports': {'attributes': {'acc_booking': {'slots': 1,
                                                          'type': 'i'},
                                          'acc_fulfillment': {'slots': 1,
                                                              'type': 'i'},
                                          'acc_shortage': {'slots': 1

# Interaction with the environment

Before starting interaction with the environment, we need to know DecisionEvent and Action first.

## DecisionEvent

Once the environment need the agent's response to promote the simulation, it will throw an **DecisionEvent**. In the scenario of ECR, the information of each DecisionEvent is listed as below:
- **tick**: (int) the corresponding tick
- **port_idx**: (int) the id of the port/agent that needs to respond to the environment
- **vessel_idx**: (int) the id of the vessel/operation object of the port/agnet.
- **snapshot_list**: (int) **Snapshots of the environment to input into the decision model** TODO: confirm the meaning
- **action_scope**: **Load and discharge scope for agent to generate decision**
- **early_discharge**: **Early discharge number of corresponding vessel**

## Action

Once we get a DecisionEvent from the envirionment, we should respond with an Action. Valid Action could be:

- None, which means do nothing.
- A valid Action instance, including:
   - **vessel_idx**: (int) the id of the vessel/operation object of the port/agent.
   - **port_idx**: (int) the id of the port/agent that take this action.
   - **quantity**: (int) the sign of this value denotes different meanings:
      - positive quantity means unloading empty containers from vessel to port.
      - negative quantity means loading empty containers from port to vessel.

## Generate random actions based on the DecisionEvent

The demo code in the Quick Start part has shown an interaction mode that doing nothing(responding with None action). Here we read the detailed information about the DecisionEvent and generate random actions based on it.

In [4]:
from maro.simulator import Env
from maro.simulator.scenarios.ecr.common import Action, DecisionEvent

import random

env = Env(scenario="ecr", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

metrics: object = None
decision_event: DecisionEvent = None
is_done: bool = False

metrics, decision_event, is_done = env.step(None)

while not is_done:
    # Generate a random Action according to the action_scope in DecisionEvent
    action = Action(
        vessel_idx=decision_event.vessel_idx,
        port_idx=decision_event.port_idx,
        quantity=random.randint(-decision_event.action_scope.load, decision_event.action_scope.discharge)
    )
    # random sampling some records to show in the output
    if random.random() > 0.95:
        print(f"*************\n{decision_event}\n{action}")
    metrics, decision_event, is_done = env.step(action) 

*************
DecisionEvent(tick=56, port_idx=4, vessel_idx=4, action_scope=ActionScope {load: 5485, discharge: 190 })
Action {quantity: -272, port: 4, vessel: 4 }
*************
DecisionEvent(tick=56, port_idx=0, vessel_idx=5, action_scope=ActionScope {load: 0, discharge: 76 })
Action {quantity: 74, port: 0, vessel: 5 }
*************
DecisionEvent(tick=91, port_idx=0, vessel_idx=3, action_scope=ActionScope {load: 0, discharge: 1419 })
Action {quantity: 381, port: 0, vessel: 3 }
*************
DecisionEvent(tick=91, port_idx=4, vessel_idx=5, action_scope=ActionScope {load: 12863, discharge: 1259 })
Action {quantity: -463, port: 4, vessel: 5 }


## Get the environment observation

You can also implement other strategies or build models to take action. At this time, real-time information and historical records of the environment are very important for making good decisions. In this case, the the environment snapshot list is exactly what you need.

The information in the snapshot list is indexed by 3 dimensions:
- A frame index or a frame index list. (int or list of int) Empty indicates for all time slides till now
- A station id (list). (int of list of int) Empty indicates for all ports/agents
- An attribute name (list). (str of list of str) You can get all available attributes in env.summary as shown before.

The return value from the snapshot list is a numpy.ndarray with shape **(frame * station * attribute, )**.

More detailed introduction to the snapshot list is [here](docs/_build/html/key_components/data_model.html#advanced-features).     # TODO: add hyper-link

In [5]:
from maro.simulator import Env
from pprint import pprint

env = Env(scenario="ecr", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

pprint(env.summary)

{'node_detail': {'matrices': {'attributes': {'full_on_ports': {'slots': 25,
                                                               'type': 'i'},
                                             'full_on_vessels': {'slots': 30,
                                                                 'type': 'i'},
                                             'vessel_plans': {'slots': 30,
                                                              'type': 'i'}},
                              'number': 1},
                 'ports': {'attributes': {'acc_booking': {'slots': 1,
                                                          'type': 'i'},
                                          'acc_fulfillment': {'slots': 1,
                                                              'type': 'i'},
                                          'acc_shortage': {'slots': 1,
                                                           'type': 'i'},
                                          'booking': {'slo

In [6]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for ECR scenario
env = Env(scenario="ecr", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

# Run the environment to the end
_, _, is_done = env.step(None)
while not is_done:
    _, _, is_done = env.step(None)

# Get the capacity info for each ports by directly using initial frame index and attribute name
capacity = env.snapshot_list["ports"][0::"capacity"]
print(type(capacity), capacity.shape)
for i in range(len(env.agent_idx_list)):
    print(f"Port {i} capacity: {capacity[i]}")
print()
    
# Get fulfillment and shortage info simultaneously by using attribute list
attribute_list = ["fulfillment", "shortage"]
info = env.snapshot_list["ports"][::attribute_list]
print(type(info), info.shape)

# Reshape the info of fulfillment and shortage into a user-friendly shape
num_attributes = len(attribute_list)
num_frame = env.frame_index + 1
num_ports = len(env.agent_idx_list)
info = info.reshape(num_frame, num_ports, num_attributes)
print(type(info), info.shape)

# Pring and show the change of shortage in each port:
shortage_idx = 1
for port_id in env.agent_idx_list:
    print(f"Port {port_id}: {info[:, port_id, shortage_idx]}")

<class 'numpy.ndarray'> (5,)
Port 0 capacity: 100000.0
Port 1 capacity: 100000.0
Port 2 capacity: 100000.0
Port 3 capacity: 100000.0
Port 4 capacity: 100000.0

<class 'numpy.ndarray'> (1000,)
<class 'numpy.ndarray'> (100, 5, 2)
Port 0: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500.]
Port 1: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 500. 500.
 500. 500. 500. 500. 500. 500. 500